In [ ]:
!git clone https://github.com/microsoft/qlib.git
!python3 qlib/setup.py install
!python3 -m qlib.run.get_data qlib_data --target_dir ~/.qlib/qlib_data/cn_data --region cn

In [2]:
from qlib.data.dataset import DatasetH, TSDatasetH
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, TensorDataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import time
import datetime
import random
import warnings

from tqdm.auto import tqdm
from Layers import FeatureExtractor, FactorEncoder, AlphaLayer, BetaLayer, FactorDecoder, FactorVAE, FactorPredictor
from stockdata import StockDataset
from tqdm.auto import tqdm
from dataclasses import dataclass
from utils import test_args, load_model, generate_prediction_scores

warnings.simplefilter("ignore", category=RuntimeWarning)
pd.set_option('display.max_rows', 500)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# %load_ext autoreload
# %autoreload 2

# Create Data

In [ ]:
# Ideal but not runnable due to current setup
# !python3 make_dataset.py --save_dir './trial1_data' --start_time '2012-01-01' --end_time '2023-11-30' --fit_end_time '2020-12-31' --val_start_time '2021-01-01' --val_end_time '2022-12-31' --seq_len 21 --normalize --select_feature

# Real commands
!python3 main.py --data_dir 'trial0_norm_data' --run_name 'lr0d0005_hz20_nl20_nf8' --save_dir './trial0_norm_models_lr0d0005_hz20_nl20_nf8' --wandb --normalize --num_epochs 5 --lr 0.0005 --hidden_size 20 --num_latent 20 --num_factor 8
!python3 main.py --data_dir 'trial0_norm_data' --run_name 'lr0d0005_hz20_nl20_nf20' --save_dir './trial0_norm_models_lr0d0005_hz20_nl20_nf20' --wandb --normalize --num_epochs 5 --lr 0.0005 --hidden_size 20 --num_latent 20 --num_factor 20
!python3 main.py --data_dir 'trial0_norm_data' --run_name 'lr0d0005_hz32_nl20_nf8' --save_dir './trial0_norm_models_lr0d0005_hz32_nl20_nf8' --wandb --normalize --num_epochs 5 --lr 0.0005 --hidden_size 32 --num_latent 20 --num_factor 8
!python3 main.py --data_dir 'trial0_norm_data' --run_name 'lr0d0005_hz64_nl20_nf8' --save_dir './trial0_norm_models_lr0d0005_hz64_nl20_nf8' --wandb --normalize --num_epochs 5 --lr 0.0005 --hidden_size 64 --num_latent 20 --num_factor 8

# Train

!python3 main.py --data_dir 'trial0_data' --run_name 'trial0' --save_dir './trial0_models' --wandb

!ython3 main.py --num_epochs 50 --lr 0.0005 --batch_size 256 --num_latent 20 --seq_len 21 --num_factor 8 --hidden_size 20 --seed 45 --run_name 'trial1' --save_dir './trial1_models' --wandb --normalize

# Profile

In [ ]:
from torch.profiler import profile, record_function, ProfilerActivity

with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA], profile_memory=True, record_shapes=True):
    with record_function("model_inference"):
        output_words = evaluate(encoder, decoder, searcher, voc, input_sentence)